In [1]:
# Importar librerias

import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# Configuracion

# Función para descargar datos históricos
def descargar_datos(tickers, inicio, fin):
    datos = yf.download(tickers, start=inicio, end=fin)['Close']
    return datos

# Función para calcular retornos
def calcular_retornos(datos):
    return datos.pct_change().dropna()

# Función para calcular el retorno total
def retorno_total(retornos):
    return (retornos + 1).prod() - 1

# Función para calcular el retorno anualizado
def retorno_anualizado(retornos, dias_por_año=252):
    return (retornos.mean() * dias_por_año)

# Función para calcular el Value at Risk (VaR)
def calcular_var(retornos, confianza=0.95):
    return np.percentile(retornos, 100 * (1 - confianza))

# Función para calcular el Ratio de Sharpe
def ratio_sharpe(retornos, tasa_libre_riesgo=0, dias_por_año=252):
    exceso_retorno = retornos - tasa_libre_riesgo / dias_por_año
    return (exceso_retorno.mean() / retornos.std()) * np.sqrt(dias_por_año)

# Función para calcular el Ratio de Sortino
def ratio_sortino(retornos, tasa_libre_riesgo=0, dias_por_año=252):
    retorno_medio = retornos.mean() * dias_por_año
    desviacion_bajista = retornos[retornos < 0].std() * np.sqrt(dias_por_año)
    return (retorno_medio - tasa_libre_riesgo) / desviacion_bajista

# Función para calcular el Ratio de Treynor
def ratio_treynor(retornos_cartera, retornos_mercado, beta, tasa_libre_riesgo=0, dias_por_año=252):
    exceso_retorno = retornos_cartera.mean() * dias_por_año - tasa_libre_riesgo
    return exceso_retorno / beta

# Función para calcular Alpha
def calcular_alpha(retornos_cartera, retornos_mercado, beta, tasa_libre_riesgo=0, dias_por_año=252):
    exceso_retorno_cartera = retornos_cartera.mean() * dias_por_año - tasa_libre_riesgo
    exceso_retorno_mercado = retornos_mercado.mean() * dias_por_año - tasa_libre_riesgo
    return exceso_retorno_cartera - (beta * exceso_retorno_mercado)

# Función para calcular Beta
def calcular_beta(retornos_cartera, retornos_mercado):
    # Asegurar que tienen las mismas fechas
    datos_comunes = retornos_cartera.index.intersection(retornos_mercado.index)
    retornos_cartera = retornos_cartera.loc[datos_comunes]
    retornos_mercado = retornos_mercado.loc[datos_comunes]

    # Calcular covarianza y beta
    covarianza = np.cov(retornos_cartera, retornos_mercado)[0, 1]
    varianza_mercado = np.var(retornos_mercado)
    return covarianza / varianza_mercado


# Función para calcular el Índice de Diversificación
def indice_diversificacion(retornos_cartera, retornos_activos):
    correlaciones = retornos_activos.corr()
    diversificacion = 1 - np.mean(correlaciones)
    return diversificacion

# Función para calcular el Tracking Error
def tracking_error(retornos_cartera, retornos_benchmark):
    diferencia = retornos_cartera - retornos_benchmark
    return np.std(diferencia) * np.sqrt(252)

# Función para calcular el Information Ratio
def information_ratio(retornos_cartera, retornos_benchmark):
    diferencia = retornos_cartera - retornos_benchmark
    return diferencia.mean() / diferencia.std() * np.sqrt(252)

# Función para calcular métricas de una cartera
def calcular_metricas_cartera(retornos_cartera, retornos_benchmark, retornos_activos, tasa_libre_riesgo=0):
    beta = calcular_beta(retornos_cartera, retornos_benchmark)
    metrics = {
        'Retorno Total': retorno_total(retornos_cartera),
        'Retorno Anualizado': retorno_anualizado(retornos_cartera),
        'Retorno Relativo al S&P 500': retorno_total(retornos_cartera) - retorno_total(retornos_benchmark),
        'Value at Risk (95%)': calcular_var(retornos_cartera),
        'Ratio de Sharpe': ratio_sharpe(retornos_cartera, tasa_libre_riesgo),
        'Ratio de Sortino': ratio_sortino(retornos_cartera, tasa_libre_riesgo),
        'Ratio de Treynor': ratio_treynor(retornos_cartera, retornos_benchmark, beta, tasa_libre_riesgo),
        'Alpha': calcular_alpha(retornos_cartera, retornos_benchmark, beta, tasa_libre_riesgo),
        'Beta': beta,
        'Índice de Diversificación': indice_diversificacion(retornos_cartera, retornos_activos),
        'Tracking Error': tracking_error(retornos_cartera, retornos_benchmark),
        'Information Ratio': information_ratio(retornos_cartera, retornos_benchmark)
    }
    return metrics

# Función principal para comparar múltiples carteras
def comparar_carteras(lista_carteras, inicio, fin, tasa_libre_riesgo=0):
    # Descargar datos del benchmark (S&P 500)
    datos_benchmark = descargar_datos(['^GSPC'], inicio, fin)
    retornos_benchmark = calcular_retornos(datos_benchmark)['^GSPC']

    # DataFrame para almacenar resultados
    resultados = pd.DataFrame()

    for i, cartera in enumerate(lista_carteras):
        tickers, ponderaciones = cartera
        datos = descargar_datos(tickers, inicio, fin)
        retornos = calcular_retornos(datos)
        retornos_cartera = (retornos[tickers] * ponderaciones).sum(axis=1)

        # Asegurar que los retornos de la cartera y el benchmark tienen las mismas fechas
        datos_comunes = retornos_cartera.index.intersection(retornos_benchmark.index)
        retornos_cartera = retornos_cartera.loc[datos_comunes]
        retornos_benchmark = retornos_benchmark.loc[datos_comunes]

        # Calcular métricas
        metricas = calcular_metricas_cartera(retornos_cartera, retornos_benchmark, retornos[tickers], tasa_libre_riesgo)
        resultados[f'Cartera {i+1}'] = pd.Series(metricas)

    return resultados

In [3]:
# Carteras

if __name__ == "__main__":
    # Definir las carteras (tickers y ponderaciones)
    lista_carteras = [
        (['AAPL', 'MSFT', 'GOOGL', 'AMZN'], [0.25, 0.25, 0.25, 0.25]),  # Cartera 1
        (['TSLA', 'NVDA', 'AMD', 'INTC'], [0.3, 0.3, 0.2, 0.2]),        # Cartera 2
        (['BRK-B', 'JNJ', 'PG', 'KO'], [0.4, 0.3, 0.2, 0.1]),          # Cartera 3
        (['BTC-USD', 'ETH-USD', 'LTC-USD'], [0.5, 0.3, 0.2]),          # Cartera 4
        (['GLD', 'SLV', 'IAU'], [0.6, 0.3, 0.1])                       # Cartera 5
    ]

    inicio = '2018-01-01'
    fin = '2023-01-01'
    tasa_libre_riesgo = 0.02  # 2% de tasa libre de riesgo

    # Comparar carteras
    resultados = comparar_carteras(lista_carteras, inicio, fin, tasa_libre_riesgo)

resultados

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


,Cartera 1,Cartera 2,Cartera 3,Cartera 4,Cartera 5
Retorno Total,1.290018,3.064532,0.659774,-0.466857,0.385414
Retorno Anualizado,0.208894,0.375794,0.118605,0.140795,0.080906
Retorno Relativo al S&P 500,0.865771,2.640285,0.235527,-0.891104,-0.038833
Value at Risk (95%),-0.030771,-0.044120,-0.016454,-0.068894,-0.017361
Ratio de Sharpe,0.645633,0.819205,0.533255,0.167576,0.345119
Ratio de Sortino,0.862761,1.152736,0.669159,0.219771,0.467796
Ratio de Treynor,0.161135,0.235516,0.140589,0.115527,0.398008
Alpha,0.101117,0.242676,0.046088,0.042503,0.049448
Beta,1.172275,1.510700,0.701370,1.045600,0.153027
Índice de Diversificación,0.208426,0.366381,0.310582,0.125486,0.096832
